In [1]:
import torch
import torch.nn as nn

In [ ]:
def apply(self, fn):#多叉树式递归查找
    for module in self.children():
        module.apply(fn)#递归入口,相当于
    for param in self.parameters():
        if param is not None:
            param.data = fn(param.data)
            if param.grad is not None:
                param.grad.data = fn(param.grad.data)
def train(self, mode=True):#此处设为True
    self.training = mode#实际执行的函数,相当于将每个叶子节点training标志设为True
    for child in self.children():
        child.train(mode)#

## Module 源代码
下面是pytorch中的源代码，我们对其中部分内容进行了注释

In [ ]:

class Module(object):
    def __init__(self):
        self._parameters = OrderedDict()
        self.training = True
        self._modules = OrderedDict()#初始化的模块全部以字典的形式存储,变量声明为protected类型
        self._buffers = OrderedDict()
        self._backward_hooks = OrderedDict()
        self._forward_pre_hooks = OrderedDict()
    def resiger_buffer(self, name, tensor):
        if not isinstance(name, torch._six.string_classes):
            raise TypeError("Buffer name should be a string")
        elif '.' in name:
            raise KeyError("Buffer name can't be empty")
        else:
            self._buffers[name] = tensor
    def register_parameter(self, name, param):
        if '_parameters' not in self.__dict__:#self.__dict__判断名称是否在self的变量中
            raise AttributeError(#善用raise提出error
            "cannot assign parameter before Module.__init__() call")
        elif not isinstance(name, torch._six.string_classes):
            raise 
    def cuda(self, device=None):
        return self._apply(lambda t: t.cuda(device))
    def add_module(self, name, module):
        if not isinstance(module, Module) and module is not None:#判断添加的是否Module对象
            raise TypeError('Error')
        elif not isinstance(name, torch._six.string_classes):
            raise TypeError("module name cannot contain \".\"")
        self._modules[name] = module
    def _apply(self, fn):
        for module in self.children():#给子Module应用函数,递归的调用,相当于多叉树,先进性位置定位,使用for循环递归式遍历
            module._apply(fn)
        for param in self._parameters.values():
            if param is not None:
                param.data = fn(param.data)
                if param._grad is not None:
                    param._grad.data = fn(param._grad.data)
    def apply(self, fn):
        for module in self.children():
            module.apply(fn)
        fn(self)
        return self
    
    def children(self):
        for name, module in self.named_children():#使用生成器的方式逐个产出数据
            yield module
    def named_children(self):
        memo = set()
        for name, module in self._modules.items():#对字典进行遍历,构造一个生成器,yield返回
            if module is not None and module not in memo:
                memo.add(module)
                yield name, module
    def modules(self):
        r"""
        返回一个所有模块的迭代器
        """
        for name, module in self.named_modules():
            yield module
    def named_modules(self, memo=None, prefix=''):
        if memeo is None:
            memo = set()
        if self not in memo:
            memo.add(self)
            yield prefix, self
            for name, module in self._modules.items():
                if module is None:
                    continue
                submodule_prefix = prefix + ('.' if prefix else '') + name
                for m in module.named_modules(memo, submodule_prefix):
                    yield m
    def train(self, mode=True):
        r"""
        只对某几层模型有效:Dropout, BatchNorm
        """
        self.training = mode
        for module in self.children():
            module.train(mode)
    def eval(self):
        return self.train(False)
    def zero_grad(self):
        for p in self.parameters():
            if p.grad is not None:
                p.grad.detach_()#分离出某个节点,使其运算不会影响梯度
                p.grad.zero_()
    def forward(self, *input):
        raise NotImplementedError
    def __call__(self, *input, **kwargs):#在call中会调用forward函数,因此model的call传入了实参,和forward的形参保持一致
        for hook in self._forward_pre_hooks.values():
            hook(self, input)
        if torch._c._get_tracing_state():
            result = self._slow_forward(*input, **kwargs)
        else:
            result = self.for
    def register_forward_pre_hook(self, hook):
        handle = hooks.RemovableHandle(self._forward_pre_hooks)
        self._forward_pre_hooks[handle.id] = hook

In [2]:
class A:
    def __init__(self):
        print('I am a initer')
    def __call__(self, *input):
        retult = self.forward(*input)
    def forward(self, v1, v2):
        print(v1, v2)